# DRAFT: Accessing ICESat-2 Data

This notebook covers ICESat-2 data query and access from the NASA NSIDC DAAC (NASA National Snow and Ice Data Center Distributed Active Archive Center). Specifically, it illustrates a recommended pathway to obtain data programmatically using the Python library `icepyx`.

#### Credits
* notebook by: Jessica Scheick and Amy Steiker
* source material: [NSIDC DAAC data access with icepyx](https://github.com/icesat2py/icepyx/blob/master/doc/examples/ICESat-2_DAAC_DataAccess_Example.ipynb) by Jessica Scheick


## Objectives

(draft)
* big data landscape
* programmatic ICESat-2 data access

## Tutorial Setup

Open a terminal in Jupyterlab ("+" icon in upper left corner, then click on the terminal icon)

1. Navigate to local directory where you want to store tutorial material:  
`cd ~`

2. Clone the repo:  
`git clone https://github.com/ICESAT-2HackWeek/data-access.git`

3. Enter the tutorial directory:  
`cd data-access`

## Ways to explore and obtain ICESat-2 data

There are numerous interactive, web-based, command line, and programmatic ways to access ICESat-2 data...

![ICESat-2 Data Access Options](https://github.com/ICESAT-2HackWeek/data-access/supporting_docs/IS2_data_access_options_d3.png)


#### OpenAltimetry

https://openaltimetry.org/

Some info about it here...
An exploratory exercise (so maybe have some questions here? In which case maybe make this its own notebook since it will be a different day than the programmatic access part?)

## NSIDC DAAC Programmatic Data Access using icepyx

### Import packages, including icepyx

In [ ]:
from icepyx import icesat2data as ipd
import os
import shutil
%matplotlib inline

Remove the next two cells once the notebook is developed/tested

In [ ]:
%cd ./../

In [ ]:
%load_ext autoreload
from icepyx import icesat2data as ipd
%autoreload 2
#in order to use "as ipd", you have to use autoreload 2, which will automatically reload any module not excluded by being imported with %aimport -[module]

### Key Steps for Programmatic Data Access

There are several key steps for accessing data from the NSIDC API:
1. Define your parameters (spatial, temporal, dataset, etc.)
2. Log in to Earthdata
3. Define additional parameters (e.g. subsetting/customization options) (Tuesday's tutorial)
4. Order your data
5. Download your data

We'll go through each of these steps during this tutorial, at the end summarizing how icepyx streamlines this process into a minimal number of lines of code.

### Step 1: Create an ICESat-2 data object with the desired search parameters

There are three required inputs:
- `short_name` = the dataset of interest, known as its "short name".
See https://nsidc.org/data/icesat-2/data-sets for a list of the available datasets.
- `spatial extent` = a region of interest to search within. This can be entered as a bounding box, polygon vertex coordinate pairs, or a polygon geospatial file (currently shp, kml, and gpkg are supported, though gpkg is still untested).
    - bounding box: Given in decimal degrees for the lower left longitude, lower left latitude, upper right longitude, and upper right latitude
    - polygon vertices: Given as longitude, latitude coordinate pairs of decimal degrees with the last entry a repeat of the first.
    - polygon file: A string containing the full file path and name.
- `date_range` = the date range for which you would like to search for results. Must be formatted as a set of 'YYYY-MM-DD' strings.

Below are examples of multiple spatial extent inputs, which provide the code to download the data used throughout the Hackweek. Although you may choose to run any of the locations in the next three cells, we will be using ____.

In [ ]:
#Byrd Glacier/Ross Ice Shelf
short_name = 'ATL06'
spatial_extent = [148, -81, 162, -80]
date_range = ['2018-10-14','2018-12-22']

In [ ]:
#Arctic Ocean
short_name = 'ATL07'
spatial_extent = [0, 80, 180, 90]
date_range = ['2019-02-01','2019-03-01']

In [ ]:
#Northwest Continental US
short_name = 'ATL06'
spatial_extent = [-125, 40, -120, 50]
date_range = ['2019-06-01','2019-07-01']

Create the data object using our inputs

In [ ]:
region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range)

Formatted parameters and function calls allow us to see the the properties of the data object we have created.

In [ ]:
print(region_a.dataset)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.dataset_version)
print(region_a.spatial_extent)
region_a.visualize_spatial_extent()

There are also several optional inputs to allow the user finer control over their search.
- `start_time` = start time to search for data on the start date. If no input is given, this defaults to 00:00:00.
- `end_time` = end time for the end date of the temporal search parameter. If no input is given, this defaults to 23:59:59. Times must be input as 'HH:mm:ss' strings.
- `version` = What version of the dataset to use, input as a numerical string. If no input is given, this value defaults to the most recent version of the dataset specified in `short_name`.

*NOTE* Version 001 is used as an example in the below cell. However, using it will cause 'no results' errors in granule ordering for some search parameters. These issues have been resolved in later versions of the datasets, so it is best to use the most recent version where possible. Thus, you will need to change the version associated with `region_a` and rerun the next cell for the rest of this notebook to run.

In [ ]:
region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range, \
                           start_time='03:30:00', end_time='21:30:00', version='001')

print(region_a.dataset)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.dataset_version)
print(region_a.spatial_extent)

Alternatively, you can also just create the data object without creating named variables first:

In [ ]:
# region_a = ipd.Icesat2Data('ATL06',[-55, 68, -48, 71],['2019-02-01','2019-02-28'], \
#                            start_time='00:00:00', end_time='23:59:59', version='002')

#### Built in methods allow us to get more information about our dataset
In addition to viewing the stored object information shown above (e.g. dataset, start and end date and time, version, etc.), we can also request summary information about the dataset itself or confirm that we have manually specified the latest version.

In [ ]:
region_a.dataset_summary_info()
print(region_a.latest_version())

If the summary does not provide all of the information you are looking for, or you would like to see information for previous versions of the dataset, all available metadata for the collection dataset is available in a readable format.

In [ ]:
region_a.dataset_all_info()

#### Querying a dataset
In order to search the dataset collection for available data granules, we need to build our search parameters. This is done automatically behind the scenes when you run `region_a.avail_granules()`, but you can also build and view them by calling `region_a.CMRparams`. These are formatted as a dictionary of key:value pairs according to the CMR documentation.

In [ ]:
#build and view the parameters that will be submitted in our query
region_a.CMRparams

Now that our parameter dictionary is constructed, we can search the CMR database for the available granules. Granules returned by the CMR metadata search are automatically stored within the data object. The search completed at this level relies completely on the granules' metadata. As a result, some (and in rare cases all) of the granules returned may not actually contain data in your specified region, particularly if the region is small or located near the boundaries of a given granule. If this is the case, the subsetter will not return any data when you actually place the order.

In [ ]:
#search for available granules and provide basic summary info about them
region_a.avail_granules()

In [ ]:
#print detailed information about the returned search results
region_a.granules.avail

### Step 2: Log in to Earthdata
In order to download any data from NSIDC, we must first authenticate ourselves using a valid Earthdata login. This will create a valid token to interface with the DAAC as well as start an active logged-in session to enable data download. Once you have successfully logged in for a given icesat2data instance, the token and session will be passed behind the scenes as needed for you to order and download data. Passwords are entered but not shown or stored in plain text by the system.

In [ ]:
earthdata_uid = 'icepyx_devteam'
email = 'icepyx.dev@gmail.com'
region_a.earthdata_login(earthdata_uid, email)

### Step 3: Additional Parameters

Once we have generated our session, we must build the required configuration parameters needed to actually download data. These will tell the system how we want to download the data. As with the CMR search parameters, these will be built automatically when you run `region_a.order_granules()`, but you can also create and view them with `region_a.reqparams`. The default parameters, given below, should work for most users.
- `page_size` = 10. This is the number of granules we will request per order.
- `page_num` = 1. Determine the number of pages based on page size and the number of granules available. If no page_num is specified, this calculation is done automatically to set page_num, which then provides the number of individual orders we will request given the number of granules.
- `request_mode` = 'async'
- `agent` = 'NO'
- `include_meta` = 'Y'

#### More details about the configuration parameters
`request_mode` is "synchronous" by default, meaning that the request relies on a direct, continous connection between you and the API endpoint. Outputs are directly downloaded, or "streamed", to your working directory. For this tutorial, we will set the request mode to asynchronous, which will allow concurrent requests to be queued and processed without the need for a continuous connection.

**Use the streaming `request_mode` with caution: While it can be beneficial to stream outputs directly to your local directory, note that timeout errors can result depending on the size of the request, and your request will not be queued in the system if NSIDC is experiencing high request volume. For best performance, NSIDC recommends setting `page_size=1` to download individual outputs, which will eliminate extra time needed to zip outputs and will ensure faster processing times per request.**

Recall that we queried the total number and volume of granules prior to applying customization services. `page_size` and `page_num` can be used to adjust the number of granules per request up to a limit of 2000 granules for asynchronous, and 100 granules for synchronous (streaming). For now, let's select 9 granules to be processed in each zipped request. For ATL06, the granule size can exceed 100 MB so we want to choose a granule count that provides us with a reasonable zipped download size. 

In [ ]:
print(region_a.reqparams)
# region_a.reqparams['page_size'] = 9
# print(region_a.reqparams)

#### Additional Parameters: Subsetting (more details provided on Tuesday)

In addition to the required parameters (CMRparams and reqparams) that are submitted with our order, for ICESat-2 datasets we can also submit subsetting parameters to NSIDC. This utilizes the NSIDC's built in subsetter to extract only the data you are interested (spatially, temporally, variables of interest, etc.). The advantages of using the NSIDC's subsetter include:
* easily reproducible downloads, particularly when coupled with and icepyx data object
* smaller file size, meaning faster downloads, less storage required, and no need to subset the data on your own
* still easy to go back and order more data/variables with the same or similar search parameters
* no extraneous data means you can move directly to analysis and easily navigate your dataset

Certain subset parameters are specified by default unless `subset=False` is included as an input to `order_granules()`. A separate, companion notebook tutorial covers subsetting in more detail, including how to get a list of subsetting options, how to build your list of subsetting parameters, and how to generate a list of desired variables (most datasets have more than 200 variable fields!), including using pre-built default lists (these lists are still in progress and we welcome contributions!).

As for the CMR and required parameters, default subset parameters can be built and viewed using `subsetparams`. Where an input spatial file is used, rather than a bounding box or manually entered polygon, the spatial file will be used for subsetting (unless subset is set to False) but not show up in the `subsetparams` dictionary.

In [ ]:
region_a.subsetparams

In [ ]:
region_a._geom_filepath

### Step 4: Place the Data Order
Then, we can send the order to NSIDC using the order_granules function. Information about the granules ordered and their status will be printed automatically as well as emailed to the address provided. Additional information on the order, including request URLs, can be viewed by setting the optional keyword input 'verbose' to True.

In [ ]:
region_a.order_granules()
# region_a.order_granules(verbose=True)

In [ ]:
#view a short list of order IDs
region_a.granules.orderIDs

### Step 5: Download the Order
Finally, we can download our order to a specified directory (which needs to have a full path but doesn't have to point to an existing directory) and the download status will be printed as the program runs. Additional information is again available by using the optional boolean keyword `verbose`.

In [ ]:
path = './download'
region_a.download_granules(path)

#### Clean up the download folder by removing individual order folders:

In [ ]:
#Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
        
for root, dirs, files in os.walk(path):
    for name in dirs:
        os.rmdir(os.path.join(root, name))

### Quick-Start Quide
The entire process of getting ICESat-2 data (from query to download) can ultimately be accomplished in three minimal lines of code:

`region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range)`

`region_a.earthdata_login(earthdata_uid, email)`

`region_a.download_granules(path)`

where the function inputs are described throughout this tutorial.

**This minimal example does not include non-default (spatial and temporal) subsetting and customization, which will be covered in more detail in Tuesday's tutorial.** The motivation for using some of the more detailed steps outlined in this notebook are that they provide the user with much more control over the data they download, ultimately saving time and effort later on in the processing and storage pipelines.